In [ ]:
import pandas as pd

# This script reads main_for_tableau.csv and creates current_snapshot.csv
FULL_HISTORICAL_DATA_FILE = 'main_for_tableau.csv' # This file is generated by the previous Python script

full_data = pd.read_csv(FULL_HISTORICAL_DATA_FILE)

# Ensure snapshot_date is in datetime format for proper sorting
full_data['snapshot_date'] = pd.to_datetime(full_data['snapshot_date'])

# --- Step 1: Get the latest snapshot for EACH unique employee who ever appeared in the data ---
# Sort by employee_id first, then by snapshot_date in descending order.
full_data_sorted = full_data.sort_values(by=['employee_id', 'snapshot_date'], ascending=[True, False])

# Drop duplicates on 'employee_id' while keeping the 'first' (which is the latest after sorting)
latest_per_employee_data_all = full_data_sorted.drop_duplicates(subset=['employee_id'], keep='first').copy()

# --- Step 2: Filter this comprehensive "latest snapshot" to include ONLY currently ACTIVE employees ---
# # This relies on 'ever_terminated_flag' being present in main_for_tableau.csv and being 0 for active
# current_snapshot_active_only = latest_per_employee_data_all[
#     latest_per_employee_data_all['ever_terminated_flag'] == 0
# ].copy()

# # --- Export the filtered data to 'current_snapshot.csv' ---
# current_snapshot_active_only.to_csv('current_snapshot.csv', index=False)
# print(f"Exported 'current_snapshot.csv' with the final snapshot for each **ACTIVE** employee only.")

# Define a recency threshold for "active" snapshots (e.g., within the last year, or since a specific date)
# Let's use 2024-01-01 as an example threshold for "recent" snapshots. Adjust as needed.
recent_snapshot_threshold = pd.to_datetime('2025-01-01')

# Filter for actual active employees AND whose latest snapshot is recent enough
current_snapshot = latest_per_employee_data_all[
    (latest_per_employee_data_all['ever_terminated_flag'] == 0) & # Still marked as active
    (latest_per_employee_data_all['snapshot_date'] >= recent_snapshot_threshold) # And their last snapshot is recent
].copy()

# Export this filtered data to a new CSV file
current_snapshot.to_csv('current_snapshot.csv', index=False)
print(f"Exported 'current_snapshot.csv' with the final snapshot for truly current, active employees.")
print(f"Number of employees in current_snapshot.csv after recency filter: {len(current_snapshot)}")